In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv(r"C:\Users\Jonat\dat-11-15\ClassMaterial\Unit1\data\master.csv", parse_dates = ['visit_date'])

In [8]:
df = df.fillna(0)

df = df.sort_values(by = ['id', 'visit_date'])

train = df.groupby('id').apply(lambda x: x.iloc[:-15])
test  = df.groupby('id').apply(lambda x: x.iloc[-15:])

In [12]:
X_train = train.drop(['visit_date', 'visitors'], axis = 1)
y_train = train['visitors']

X_test  = test.drop(['visit_date', 'visitors'], axis = 1)
y_test  = test['visitors']

In [15]:
pipe = make_pipeline(ce.TargetEncoder(cols = ['id']),
                     ce.OrdinalEncoder(cols = ['day_of_week', 'area']), 
                     ce.OneHotEncoder(), 
                     DecisionTreeRegressor(max_depth = 6))

pipe.fit(X_train, y_train).score(X_test, y_test)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.4554864523331681

In [11]:
# and our training score
pipe.score(X_train, y_train)

0.462123182078769

In [14]:
df.nunique()

id                  829
visit_date          478
visitors            204
day_of_week           7
holiday               2
genre                14
area                103
latitude            108
longitude           108
reserve_visitors     50
dtype: int64

In [16]:
train

id visit_date  visitors  \
id                                                                       
air_00a91d42b08b08d9 166836  air_00a91d42b08b08d9 2016-07-01        35   
                     166837  air_00a91d42b08b08d9 2016-07-02         9   
                     166838  air_00a91d42b08b08d9 2016-07-04        20   
                     166839  air_00a91d42b08b08d9 2016-07-05        25   
                     166840  air_00a91d42b08b08d9 2016-07-06        29   
...                                           ...        ...       ...   
air_fff68b929994bfbd 216629  air_fff68b929994bfbd 2017-04-03         2   
                     216630  air_fff68b929994bfbd 2017-04-04         4   
                     216631  air_fff68b929994bfbd 2017-04-05         6   
                     216632  air_fff68b929994bfbd 2017-04-06         6   
                     216633  air_fff68b929994bfbd 2017-04-07         9   

                            day_of_week  holiday           genre  \
id                                                                 
air_00a91d42b08b08d9 166836      Friday        0  Italian/French   
                     166837    Saturday        0  Italian/French   
                     166838      Monday        0  Italian/French   
                     166839     Tuesday        0  Italian/French   
                     166840   Wednesday        0  Italian/French   
...                                 ...      ...             ...   
air_fff68b929994bfbd 216629      Monday        0    Bar/Cocktail   
                     216630     Tuesday        0    Bar/Cocktail   
                     216631   Wednesday        0    Bar/Cocktail   
                     216632    Thursday        0    Bar/Cocktail   
                     216633      Friday        0    Bar/Cocktail   

                                                        area   latitude  \
id                                                                        
air_00a91d42b08b08d9 166836  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     166837  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     166838  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     166839  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     166840  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
...                                                      ...        ...   
air_fff68b929994bfbd 216629        Tōkyō-to Nakano-ku Nakano  35.708146   
                     216630        Tōkyō-to Nakano-ku Nakano  35.708146   
                     216631        Tōkyō-to Nakano-ku Nakano  35.708146   
                     216632        Tōkyō-to Nakano-ku Nakano  35.708146   
                     216633        Tōkyō-to Nakano-ku Nakano  35.708146   

                              longitude  reserve_visitors  
id                                                         
air_00a91d42b08b08d9 166836  139.753595               0.0  
                     166837  139.753595               4.0  
                     166838  139.753595               0.0  
                     166839  139.753595               0.0  
                     166840  139.753595               0.0  
...                                 ...               ...  
air_fff68b929994bfbd 216629  139.666288               0.0  
                     216630  139.666288               0.0  
                     216631  139.666288               2.0  
                     216632  139.666288               8.0  
                     216633  139.666288               1.0  

[239673 rows x 10 columns]

In [22]:
train = train.reset_index(drop = True)

val   = train.groupby('id').apply(lambda x: x.iloc[-15:])
train = train.groupby('id').apply(lambda x: x.iloc[:-15])

In [23]:
X_train = train.drop(['visit_date', 'visitors'], axis = 1)
y_train = train['visitors']

X_val  = val.drop(['visit_date', 'visitors'], axis = 1)
y_val  = val['visitors']

In [32]:
pipe[-1].set_params(max_depth = 8)
pipe.fit(X_train, y_train).score(X_val, y_val)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.501557945717827

In [77]:
# function to automatically check for validation score -- useful for 
def get_val_scores(df, estimator):
    
    df = df.drop('visit_date', axis = 1)
    
    # create training and validation set
    train = df.groupby('id').apply(lambda x: x.iloc[:-15]).reset_index(drop = True)
    val   = df.groupby('id').apply(lambda x: x.iloc[-15:]).reset_index(drop = True)
    
    # create a validation & test set
    X_train, y_train = train.drop('visitors', axis = 1), train['visitors']
    X_val, y_val     = val.drop('visitors', axis = 1), val['visitors']
    
    estimator.fit(X_train, y_train)
    
    # score on the test data
    return estimator.score(X_val, y_val)

In [34]:
pipe

Pipeline(steps=[('targetencoder', TargetEncoder(cols=['id'])),
                ('ordinalencoder',
                 OrdinalEncoder(cols=['day_of_week', 'area'],
                                mapping=[{'col': 'day_of_week',
                                          'data_type': dtype('O'),
                                          'mapping': Friday       1
Saturday     2
Monday       3
Tuesday      4
Wednesday    5
Thursday     6
Sunday       7
NaN         -2
dtype: int64},
                                         {'col': 'area',
                                          'data_type': dtype('O'),
                                          'mapping': Tōkyō-to Chiyoda-ku Kudanminami           1
Tōkyō-to Minat...
Tōkyō-to Taitō-ku Higashiueno             3
Ōsaka-fu Ōsaka-shi Nakanochō              4
Ōsaka-fu Ōsaka-shi Ōhiraki                5
                                       ... 
Niigata-ken Nagaoka-shi Ōtedōri         100
Tōkyō-to Setagaya-ku Kitazawa           101
Tōkyō-to Meguro-ku

In [35]:
train

id visit_date  visitors  \
id                                                                       
air_00a91d42b08b08d9 0       air_00a91d42b08b08d9 2016-07-01        35   
                     1       air_00a91d42b08b08d9 2016-07-02         9   
                     2       air_00a91d42b08b08d9 2016-07-04        20   
                     3       air_00a91d42b08b08d9 2016-07-05        25   
                     4       air_00a91d42b08b08d9 2016-07-06        29   
...                                           ...        ...       ...   
air_fff68b929994bfbd 239653  air_fff68b929994bfbd 2017-03-19         2   
                     239654  air_fff68b929994bfbd 2017-03-20         2   
                     239655  air_fff68b929994bfbd 2017-03-21         2   
                     239656  air_fff68b929994bfbd 2017-03-22         1   
                     239657  air_fff68b929994bfbd 2017-03-23         3   

                            day_of_week  holiday           genre  \
id                                                                 
air_00a91d42b08b08d9 0           Friday        0  Italian/French   
                     1         Saturday        0  Italian/French   
                     2           Monday        0  Italian/French   
                     3          Tuesday        0  Italian/French   
                     4        Wednesday        0  Italian/French   
...                                 ...      ...             ...   
air_fff68b929994bfbd 239653      Sunday        0    Bar/Cocktail   
                     239654      Monday        1    Bar/Cocktail   
                     239655     Tuesday        0    Bar/Cocktail   
                     239656   Wednesday        0    Bar/Cocktail   
                     239657    Thursday        0    Bar/Cocktail   

                                                        area   latitude  \
id                                                                        
air_00a91d42b08b08d9 0       Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     1       Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2       Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     3       Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     4       Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
...                                                      ...        ...   
air_fff68b929994bfbd 239653        Tōkyō-to Nakano-ku Nakano  35.708146   
                     239654        Tōkyō-to Nakano-ku Nakano  35.708146   
                     239655        Tōkyō-to Nakano-ku Nakano  35.708146   
                     239656        Tōkyō-to Nakano-ku Nakano  35.708146   
                     239657        Tōkyō-to Nakano-ku Nakano  35.708146   

                              longitude  reserve_visitors  
id                                                         
air_00a91d42b08b08d9 0       139.753595               0.0  
                     1       139.753595               4.0  
                     2       139.753595               0.0  
                     3       139.753595               0.0  
                     4       139.753595               0.0  
...                                 ...               ...  
air_fff68b929994bfbd 239653  139.666288              44.0  
                     239654  139.666288               3.0  
                     239655  139.666288               7.0  
                     239656  139.666288               8.0  
                     239657  139.666288              11.0  

[227248 rows x 10 columns]

In [38]:
train = df.groupby('id').apply(lambda x: x.iloc[:-15]).reset_index(drop = True)
test  = df.groupby('id').apply(lambda x: x.iloc[-15:]).reset_index(drop = True)

In [39]:
get_val_scores(train, pipe)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.501557945717827

In [42]:
pipe[-1].set_params(max_depth = 6)

DecisionTreeRegressor(max_depth=6)

In [43]:
train

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0
...,...,...,...,...,...,...,...,...,...,...
239668,air_fff68b929994bfbd,2017-04-03,2,Monday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0
239669,air_fff68b929994bfbd,2017-04-04,4,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0
239670,air_fff68b929994bfbd,2017-04-05,6,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2.0
239671,air_fff68b929994bfbd,2017-04-06,6,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,8.0


Create 3 columns and see if they improve your validation score:

 - month
 - year
 - passage of time

In [44]:
train['month'] = train['visit_date'].dt.month
train['year']  = train['visit_date'].dt.year
train['time']  = (train['visit_date'] - train['visit_date'].min()).dt.days

In [46]:
get_val_scores(train, pipe)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.47141395726391233

In [47]:
train = train.drop(['month', 'year', 'time'], axis = 1)

In [48]:
get_val_scores(train, pipe)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.47174070918546396

In [51]:
train['yesterday'] = train.groupby('id')['visitors'].shift()

In [54]:
train['yesterday'] = train.groupby('id')['visitors'].shift()
train['two_days_ago'] = train.groupby('id')['visitors'].shift(2)
train['three_days_ago'] = train.groupby('id')['visitors'].shift(3)
train['last_week'] = train.groupby('id')['visitors'].shift(7)
train['last_month'] = train.groupby('id')['visitors'].shift(30)

train = train.bfill()

In [58]:
val_score, imp = get_val_scores(train, pipe)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [75]:
train['7_day_rolling']  = train.groupby('id')['visitors'].rolling(7).mean().shift().values
train['30_day_rolling'] = train.groupby('id')['visitors'].rolling(30).mean().shift().values
train['90_day_rolling'] = train.groupby('id')['visitors'].rolling(90).mean().shift().values

In [78]:
train = train.bfill()

In [81]:
get_val_scores(train, pipe)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.5099879409825464

In [82]:
pipe

Pipeline(steps=[('targetencoder', TargetEncoder(cols=['id'])),
                ('ordinalencoder',
                 OrdinalEncoder(cols=['day_of_week', 'area'],
                                mapping=[{'col': 'day_of_week',
                                          'data_type': dtype('O'),
                                          'mapping': Friday       1
Saturday     2
Monday       3
Tuesday      4
Wednesday    5
Thursday     6
Sunday       7
NaN         -2
dtype: int64},
                                         {'col': 'area',
                                          'data_type': dtype('O'),
                                          'mapping': Tōkyō-to Chiyoda-ku Kudanminami           1
Tōkyō-to Minat...
Tōkyō-to Taitō-ku Higashiueno             3
Ōsaka-fu Ōsaka-shi Nakanochō              4
Ōsaka-fu Ōsaka-shi Ōhiraki                5
                                       ... 
Niigata-ken Nagaoka-shi Ōtedōri         100
Tōkyō-to Setagaya-ku Kitazawa           101
Tōkyō-to Meguro-ku

In [83]:
pipe[-1].feature_importances_

array([2.82904193e-02, 8.48382974e-02, 0.00000000e+00, 0.00000000e+00,
       1.06470856e-03, 0.00000000e+00, 1.39768520e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.72479991e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.28111222e-03, 0.00000000e+00, 0.00000000e+00,
       2.09912168e-02, 0.00000000e+00, 2.58338781e-02, 8.32015359e-01,
       5.14843732e-04])

In [88]:
pipe[0].transform(train)

ValueError: Unexpected input dimension 18, expected 16

In [89]:
X_train, y_train = train.drop(['visitors', 'visit_date'], axis = 1), train['visitors']

In [94]:
train_cols = pipe[2].transform(pipe[1].transform(pipe[0].transform(X_train))).columns

In [96]:
pd.DataFrame({
    'Col': train_cols,
    'Imp': pipe[-1].feature_importances_
}).sort_values(by='Imp', ascending = False)

,Col,Imp
27,30_day_rolling,0.832015
1,day_of_week,0.084838
0,id,0.028290
26,7_day_rolling,0.025834
24,last_week,0.020991
21,yesterday,0.004281
6,genre_4,0.001398
4,genre_2,0.001065
17,area,0.000772
28,90_day_rolling,0.000515


In [98]:
df.drop(['7_day_rolling', '30_day_rolling', '90_day_rolling'], axis = 1, inplace = True)

In [100]:
train.drop(['two_days_ago', 'three_days_ago', 'last_month', '7_day_rolling', '30_day_rolling', '90_day_rolling'], axis = 1,
          inplace = True)

In [101]:
train

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,yesterday,last_week
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,35.0,35.0
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0,35.0,35.0
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,9.0,35.0
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,20.0,35.0
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,25.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...
239668,air_fff68b929994bfbd,2017-04-03,2,Monday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,2.0,4.0
239669,air_fff68b929994bfbd,2017-04-04,4,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,2.0,9.0
239670,air_fff68b929994bfbd,2017-04-05,6,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2.0,4.0,2.0
239671,air_fff68b929994bfbd,2017-04-06,6,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,8.0,6.0,5.0


In [102]:
max_depth = [5, 6, 7]
max_features = [0.6, 0.8, 1]
min_samples_leaf = [5, 20, 40]
cv_scores = []

for depth in max_depth:
    for feature in max_features:
        for sample in min_samples_leaf:
            print(f"Getting validation score for: depth: {depth}, max_features: {feature}, leaf_samples: {sample}")
            pipe[-1].set_params(max_depth = depth, max_features = feature, min_samples_leaf = sample)
            val_score = get_val_scores(train, pipe)
            cv_scores.append((val_score, depth, feature, sample))

Getting validation score for: depth: 5, max_features: 0.6, leaf_samples: 5


C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Getting validation score for: depth: 5, max_features: 0.6, leaf_samples: 20


C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Getting validation score for: depth: 5, max_features: 0.6, leaf_samples: 40


...

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Getting validation score for: depth: 7, max_features: 1, leaf_samples: 20


C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Getting validation score for: depth: 7, max_features: 1, leaf_samples: 40


C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [104]:
max(cv_scores)

(0.5017649157528723, 7, 0.8, 40)

In [106]:
pipe[-1].set_params(max_depth = 7, max_features = 0.8, min_samples_leaf = 40)

DecisionTreeRegressor(max_depth=7, max_features=0.8, min_samples_leaf=40)

In [108]:
df['yesterday'] = df.groupby('id')['visitors'].shift()
df['one_week_ago'] = df.groupby('id')['visitors'].shift(7)

df = df.bfill()

In [109]:
get_val_scores(df, pipe)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.4798922705320312